In [1]:
import pandas as pd
from midiutil import MIDIFile
import math

In [2]:
sessions = pd.read_csv('musicmile/locations_1634469751686.txt', header=None)
bluetooth = pd.read_csv('musicmile/bluetooth_1634469751686.txt', header=None)
sessions.head(10)

,0,1,2,3,4,5,6,7
0,1634469751897,52.407536,-1.493538,133.035937,0.000000,0.000000,3.0,8.280
1,1634469751904,52.407536,-1.493538,133.035937,0.000000,0.000000,3.0,8.280
2,1634469754400,52.407152,-1.493685,132.613901,0.000000,0.000000,3.0,19.398
3,1634469759629,52.407184,-1.493628,132.674673,0.013831,1.087892,3.0,19.165
4,1634469764801,52.407334,-1.493649,132.719942,109.497640,0.075676,3.0,12.685
5,1634469770321,52.407351,-1.493666,132.578359,262.857120,0.482295,3.0,20.498
6,1634469775822,52.407362,-1.493690,132.450657,256.917500,0.514782,3.0,13.614
7,1634469781106,52.407361,-1.493701,132.487889,231.943360,0.415898,3.0,11.442
8,1634469786678,52.407331,-1.493695,132.628013,277.514530,0.118943,3.0,9.694
9,1634469792628,52.407294,-1.493670,132.584428,169.584870,0.401546,3.0,8.000


In [3]:
def clean_organisation(org):
    return org.replace('{', '').replace('}','').split('=')[0]

bluetooth['company'] = bluetooth[7].apply(clean_organisation)

In [4]:
def find_start_end(session):
   notes = bluetooth[(bluetooth[0] >= session[0])&(bluetooth[0] < (session[0]+5000))]
   return notes[[7,2, 'company']]
iots = [find_start_end(session) for session in sessions.to_numpy()]

In [17]:
company_db = pd.read_csv('companies.csv', header=None)
companys = {}
for c in company_db.to_numpy():
    companys[c[0]] = c[3]
companys

{'0': 'Technology',
 '3': 'Technology',
 '6': 'Technology',
 '76': 'Technology',
 '87': nan,
 '101': 'Technology',
 '117': 'entertainment',
 '135': 'Fitness',
 '143': nan,
 '147': 'entertainment',
 '196': 'Smart',
 '224': 'Music',
 '259': nan,
 '272': nan,
 '301': 'entertainment',
 '343': 'Fitness',
 '409': 'Security',
 '432': nan,
 '637': 'Technology',
 '761': nan,
 '784': 'Smart home',
 '1023': 'Fitness',
 '1042': 'Medical',
 '1079': 'Analytics',
 '1221': nan,
 '1447': 'Music',
 '2000': 'Smart home',
 '2059': 'Smart home',
 '2561': 'Storage',
 '2817': nan,
 '3073': 'Automotive',
 '3787': 'Music',
 '3843': 'Technology',
 '4780': nan,
 '9474': 'entertainment',
 '12360': 'Smart home',
 '14129': nan,
 '16896': 'Music',
 '17740': 'Music',
 '19456': nan,
 '24576': 'Fitness',
 '25723': nan,
 '26735': nan,
 '33406': 'Technology',
 '34817': 'Smart home',
 '34818': nan,
 '34895': nan,
 '35923': nan,
 '36967': nan,
 '37995': nan,
 '38189': nan,
 '38509': nan,
 '38524': nan,
 '40035': nan,
 '415

In [11]:
track    = 0
channel  = 0
time     = 0    # In beats
duration = 1    # In beats
tempo    = 600   # In BPM
volume   = 62  # 0-127, as per the MIDI standard
# One track, defaults to format 1 (tempo track is created automatically)
# set tracks to max number of streams connected
MyMIDI = MIDIFile(1)  
MyMIDI.addTempo(track, time, tempo)

In [22]:
sess_count = 0
for thing in iots:
    if not thing.empty:
        MyMIDI = MIDIFile(1)  
        MyMIDI.addTempo(track, time, tempo)
        beat_time = 0
        for iot in thing.to_numpy():
            beat_time += 1
            sess_count += 1
            dev_type = ''
            if iot[2] != " ":
                dev_type = companys[iot[2].strip()]
            else:
                dev_type = 'unknown'
                
            if dev_type == "Security":
                MyMIDI.addNote(0, 1, 59, beat_time, 0.2, 40)
            if dev_type == "Technology":
                MyMIDI.addNote(0, 1, 55, beat_time, 0.2, 40)
            if dev_type == "Fitness":
                MyMIDI.addNote(0, 1, 40, beat_time, 0.2, 40)
            if dev_type == "entertainment":
                MyMIDI.addNote(0, 1, 45, beat_time, 0.2, 40)
            if dev_type == "Smart home" or  dev_type == "Smart":
                MyMIDI.addNote(0, 1, 40, beat_time, 0.2, 40)
            else:
                MyMIDI.addNote(0, 1, 20, beat_time, 0.32, 60)
    with open(str(sess_count)+"_session.mid", "wb") as output_file:
        MyMIDI.writeFile(output_file)